In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import json
from datetime import datetime
import holidays
from datetime import datetime

import plotly.io as pio
import plotly.graph_objects as go
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')

pio.renderers.default = "notebook"
pio.templates.default = "plotly_white"


# this enables relative path imports
import os
from dotenv import load_dotenv
load_dotenv()
_PROJECT_PATH: str = os.environ["_project_path"]
_PICKLED_DATA_FILENAME: str = os.environ["_pickled_data_filename"]

import sys
from pathlib import Path
project_path = Path(_PROJECT_PATH)
sys.path.append(str(project_path))

import config_v2 as cfg

from library_report_v2 import Cleaning as cln
from library_report_v2 import Graphing as grp
from library_report_v2 import Processing as pro
from library_report_v2 import Configuration as repcfg

In [2]:
#Se carga los datos de produccion para toneladas
df = pd.read_excel('../data/Produccion_toneladas.xlsx')
df = df.set_index('Fecha')

#Se carga los datos de energíaP
df_energia = pd.read_pickle(project_path / 'data' / _PICKLED_DATA_FILENAME)
df_energia = df_energia.sort_values(by=['variable','datetime'])
df_energia = pro.datetime_attributes(df_energia)

#### Producción Carvajal

In [3]:
# Del df_energia se saca los dataframe por cada proceso (ESPUMA - EXTRUSORA - LINEA 7 - TERMOFORMADORA)
ea_espuma = df_energia.query("variable == 'ea-espuma'").copy()
ea_extrusora = df_energia.query("variable == 'ea-extrusora-welex'").copy()
ea_termo = df_energia.query("variable == 'ea-termoformadora'").copy()
ea_linea7 = df_energia[df_energia['variable'].isin(['ea-horno-recocido', 'ea-tubos-colapsibles'])].copy()

In [4]:
# Copio los datos de producción en un nuevo dataframe para no cambiar  la fuente original
df_producion = df.copy()
df_producion = pro.datetime_attributes(df_producion)

In [5]:
#Del dataframe producción saco 4 dataframe por cada proceso (ESPUMA - EXTRUSORA - LINEA 7 - TERMOFORMADORA)
espuma = df_producion.query("Proceso == 'Espuma'").copy()
extrusora = df_producion.query("Proceso == 'Extrusora Welex 2501'").copy()
termoformadora2 = df_producion.query("Proceso == 'Termoformadora Gabler 2'").copy()
linea7 = df_producion.query("Proceso == 'linea 7'").copy()

In [6]:
# De los df de energía los agrupo por dia, debido a que los datos de producción vienen diario

ea_espuma = ea_espuma.groupby(by=["variable"]).resample('D').sum().reset_index().set_index('datetime')
ea_espuma = pro.datetime_attributes(ea_espuma) 

ea_extrusora = ea_extrusora.groupby(by=["variable"]).resample('D').sum().reset_index().set_index('datetime')
ea_extrusora = pro.datetime_attributes(ea_extrusora) 

ea_termo = ea_termo.groupby(by=["variable"]).resample('D').sum().reset_index().set_index('datetime')
ea_termo = pro.datetime_attributes(ea_termo) 

ea_linea7 = ea_linea7.groupby(by=["variable"]).resample('D').sum().reset_index().set_index('datetime')
ea_linea7 = pro.datetime_attributes(ea_linea7) 

In [7]:
#Convertidos los index sin formato horario

ea_espuma.index = ea_espuma.index.tz_localize(None).tz_localize('America/Bogota').tz_convert(None)
ea_extrusora.index = ea_extrusora.index.tz_localize(None).tz_localize('America/Bogota').tz_convert(None)
ea_termo.index = ea_termo.index.tz_localize(None).tz_localize('America/Bogota').tz_convert(None)
ea_linea7.index = ea_linea7.index.tz_localize(None).tz_localize('America/Bogota').tz_convert(None)


espuma.index = espuma.index.tz_localize(None).tz_localize('America/Bogota').tz_convert(None)
extrusora.index = extrusora.index.tz_localize(None).tz_localize('America/Bogota').tz_convert(None)
termoformadora2.index = termoformadora2.index.tz_localize(None).tz_localize('America/Bogota').tz_convert(None)
linea7.index = linea7.index.tz_localize(None).tz_localize('America/Bogota').tz_convert(None)

In [8]:
#Solo para el proceso de Linea 7, debido a que en energía son dos maquinas para linea 7 y una sola para producción

ea_linea7['variable'] = ea_linea7['variable'].replace({'ea-horno-recocido': 'linea7', 'ea-tubos-colapsibles': 'linea7'})
# Sumar los valores de la columna "value" agrupados por el índice "datetime" y la columna "variable"
df_sum_value = ea_linea7.groupby([ea_linea7.index, 'variable'])['value'].sum()

# Restaurar el índice datetime y convertir la serie resultante en un DataFrame
df_sum_value = df_sum_value.reset_index()

# Seleccionar todas las columnas excepto la columna 'value' del DataFrame original
other_columns = ea_linea7.drop(columns=['value'])

# Unir el DataFrame de la suma de 'value' con las otras columnas del DataFrame original
linea7_sum = pd.merge(df_sum_value, other_columns, on=['datetime', 'variable'])

linea7_sum.set_index('datetime',inplace=True)

linea7_sum = linea7_sum.drop_duplicates()

#Al final trabajamos con el sgt dataframe como resultado
linea7_sum.index = linea7_sum.index.tz_localize(None).tz_localize('America/Bogota').tz_convert(None)

In [9]:
#Este paso lo hacemos solo para espuma debido en que proceso producción de espuma se encuentran 5 maquinas y en energía es solo una correspondiente a espuma

espuma['Maquina'] = espuma['Maquina'].replace({'ESPUMADO EXTRUSORA  035-045': 'Espuma', 'ESPUMADO TERMOFORMADORA F01': 'Espuma','ESPUMADO TERMOFORMADORA F02': 'Espuma', 'ESPUMADO TERMOFORMADORA F03': 'Espuma',
       'ESPUMADO TERMOFORMADORA F04 ': 'Espuma', 'ESPUMADO TERMOFORMADORA F04': 'Espuma'})


# Sumar los valores de la columna "value" agrupados por el índice "datetime" y la columna "variable"
espuma_sum = espuma.groupby([espuma.index, 'Maquina'])['Producion'].sum()

# Restaurar el índice datetime y convertir la serie resultante en un DataFrame
espuma_sum = espuma_sum.reset_index()

# Seleccionar todas las columnas excepto la columna 'value' del DataFrame original
other = espuma.drop(columns=['Producion'])

# Unir el DataFrame de la suma de 'value' con las otras columnas del DataFrame original
espuma_ok = pd.merge(espuma_sum, other, on=['Fecha', 'Maquina'])

espuma_ok.set_index('Fecha',inplace=True)

espuma_ok = espuma_ok.drop_duplicates()

#Al final trabajamos con el sgt dataframe como resultado
espuma_ok.index = espuma_ok.index.tz_localize(None).tz_localize('America/Bogota').tz_convert(None)

In [10]:
#Se coje los df de energía / producción y se les quita el formato de fecha


#Energía
linea7_sum.index = pd.to_datetime(linea7_sum.index).strftime('%Y-%m-%d')
ea_espuma.index = pd.to_datetime(ea_espuma.index).strftime('%Y-%m-%d')
ea_extrusora.index = pd .to_datetime(ea_extrusora.index).strftime('%Y-%m-%d')
ea_termo.index = pd.to_datetime(ea_termo.index).strftime('%Y-%m-%d')

#Producción
linea7.index = pd.to_datetime(linea7.index).strftime('%Y-%m-%d')
espuma_ok.index = pd.to_datetime(espuma_ok.index).strftime('%Y-%m-%d')
extrusora.index = pd .to_datetime(extrusora.index).strftime('%Y-%m-%d')
termoformadora2.index = pd.to_datetime(termoformadora2.index).strftime('%Y-%m-%d')

In [11]:
# Se calcula a los df producción los KPI 

#Liena 7:
#linea7 = linea7[linea7['year'] == 2023]
linea7['Energia'] = linea7_sum['value']/1000
linea7['KPI'] = linea7['Energia']/linea7['Producion']
linea7_c = (linea7[linea7['KPI'] < 20]).copy()

#Espuma
#espuma_ok = espuma_ok[espuma_ok['year'] == 2023]
espuma_ok['Energia'] = ea_espuma['value']/1000
espuma_ok['KPI'] = espuma_ok['Energia']/espuma_ok['Producion']
espuma_ok_c = (espuma_ok[espuma_ok['KPI'] < 1]).copy()

#Termoformadora
#extrusora = extrusora[extrusora['year'] == 2023]
extrusora['Energia'] = ea_extrusora['value']/1000
extrusora['KPI'] = extrusora['Energia']/extrusora['Producion']
extrusora_c = (extrusora[extrusora['KPI'] < 2]).copy()

#Extrusora
#termoformadora2 = termoformadora2[termoformadora2['year'] == 2023]
termoformadora2['Energia'] = ea_termo['value']/1000
termoformadora2['KPI'] = termoformadora2['Energia']/termoformadora2['Producion']
termoformadora2_c = (termoformadora2[termoformadora2['KPI'] < 1]).copy()

In [12]:
def periodos(df):
    df['periodo'] = np.where(df.index.year ==  2023, '2023', np.where(df.index.month == 1, 'Enero', np.where(df.index.month == 2, 'Febrero', np.where(df.index.month == 3, 'Marzo', np.nan))))
    return df

In [13]:
linea7_c.index = pd.to_datetime(linea7_c.index)
espuma_ok_c.index = pd.to_datetime(espuma_ok_c.index)
extrusora_c.index = pd.to_datetime(extrusora_c.index)
termoformadora2_c.index = pd.to_datetime(termoformadora2_c.index)

linea7_c = periodos(linea7_c)
espuma_ok_c = periodos(espuma_ok_c)
extrusora_c = periodos(extrusora_c)
termoformadora2_c = periodos(termoformadora2_c)

In [14]:
espuma_ok_c.reset_index(inplace=True)
extrusora_c.reset_index(inplace=True)
termoformadora2_c.reset_index(inplace=True)
linea7_c.reset_index(inplace=True)

In [15]:
espuma.reset_index(inplace=True)
extrusora.reset_index(inplace=True)
termoformadora2.reset_index(inplace=True)
linea7.reset_index(inplace=True)

In [16]:
#espuma = espuma.pivot(index='Fecha', columns='Maquina', values='Producion')
#extrusora = extrusora.pivot(index='Fecha', columns='Maquina', values='Producion')
#termoformadora2 = termoformadora2.pivot(index='Fecha',columns='Maquina', values='Producion')
#linea7 = linea7.pivot(index='Fecha', columns='Maquina', values='Producion')

In [17]:
#espuma['Produccion'] = espuma.sum(axis=1)

In [18]:
#espuma =  espuma.groupby(['Proceso','day'])['Producion'].sum()

In [19]:

# Supongamos que df es tu DataFrame con índice datetime
df_producion['festivo'] = df_producion.index.to_series().apply(lambda x: x in holidays.Colombia())

# Supongamos que df es tu DataFrame con índice datetime y la columna 'festivo'
df_producion['dow'] = df_producion.apply(lambda row: 'festivo' if row['festivo'] else row['dow'], axis=1)


# Ordenar los días de la semana en el orden deseado (por ejemplo, lunes primero)
dias_ordenados = ["lunes", "martes", "miércoles", "jueves", "viernes", "sábado", "domingo", "festivo"]
df_producion['dow'] = pd.Categorical(df_producion['dow'], categories=dias_ordenados, ordered=True)
df_producion = df_producion.sort_values(["dow"])

In [20]:
df_producion['dow'] = df_producion['dow'].astype('category')

if 'festivo' not in df_producion['dow'].cat.categories:
    df_producion['dow'] = df_producion['dow'].cat.add_categories('festivo')

# Ahora puedes asignar 'festivo' a las filas donde 'festivo' es True
df_producion.loc[df_producion['festivo'], 'dow'] = 'festivo'

In [37]:
# Agrupar por 'Proceso' y 'dow' y calcular el promedio de 'Produccion'
df_dia = df_producion.groupby(['Proceso', 'dow'])['Producion'].mean().reset_index()

# Visualizar el DataFrame resultante
print(df_dia)


                    Proceso        dow  Producion
0                    Espuma      lunes   1.526863
1                    Espuma     martes   1.746629
2                    Espuma  miércoles   1.564100
3                    Espuma     jueves   1.710576
4                    Espuma    viernes   1.828210
5                    Espuma     sábado   1.098601
6                    Espuma    domingo   0.474646
7                    Espuma    festivo   0.945594
8      Extrusora Welex 2501      lunes  11.344022
9      Extrusora Welex 2501     martes  10.474533
10     Extrusora Welex 2501  miércoles  11.194720
11     Extrusora Welex 2501     jueves   9.921310
12     Extrusora Welex 2501    viernes  10.973271
13     Extrusora Welex 2501     sábado  10.348179
14     Extrusora Welex 2501    domingo   5.973378
15     Extrusora Welex 2501    festivo   9.235819
16  Termoformadora Gabler 2      lunes   2.001384
17  Termoformadora Gabler 2     martes   2.173668
18  Termoformadora Gabler 2  miércoles   1.873450


In [35]:
df_dia_2 = df_dia.copy
# Agrupar por 'Proceso' y 'dow' y calcular el promedio de 'Produccion'
df_dia_2 = df_producion.groupby(['dow'])['Producion'].mean().reset_index().round(2)

# Visualizar el DataFrame resultante
print(df_dia_2)

         dow  Producion
0      lunes       2.67
1     martes       2.73
2  miércoles       2.67
3     jueves       2.59
4    viernes       2.82
5     sábado       2.28
6    domingo       1.26
7    festivo       2.02


In [33]:
df_maquina = df.groupby('Proceso').sum().reset_index().round(2)
df_maquina['Proceso'] = df_maquina['Proceso'].str.capitalize()
df_maquina

# Crear el gráfico de torta
fig = px.pie(df_maquina, values='Producion', names='Proceso', 
             title='Distribución de producción Máquina',
             labels={'variable': 'Máquina', 'value': 'Consumo de Energía'},
             color_discrete_sequence=px.colors.qualitative.Set1)

# Habilitar la interactividad para agregar/quitar máquinas
fig.update_traces(hoverinfo='label+percent', textinfo='value+percent', pull=[0.1]*len(df_maquina))

# Mostrar el gráfico
fig.show()

De acuerdo con los datos de producción de Carvajal, se ha identificado la producción de las siguientes máquinas: la extrusora Welez 2501, Espuma, la termoformadora Gabler 2 y la Línea 7. El acumulado de producción desde agosto de 2023 hasta mayo de 2024 se distribuye de la siguiente manera: 
- 52.4% corresponde a la extrusora Welez 2501
- 36% a Espuma
- 10% a la termoformadora Gabler 2
- 1.54% a la Línea 7.

In [24]:
df_tabla = df_producion.pivot_table(index= 'Proceso', columns='month', values='Producion', aggfunc='sum').round(2)

Dentro de los procesos analizados, se observa que el 88% de la producción total se concentra en la extrusora Welez 2501 y Espuma. Además, en el período analizado, enero de 2024 destaca como el mes con un aumento significativo en la producción, seguido por febrero de 2024.

In [25]:
df_tabla = df_producion.pivot_table(index= 'Proceso', columns='month', values='Producion', aggfunc='sum').round(1)

# Calcular los totales por columna
total_ago = df_tabla[8].sum().round()
total_sep = df_tabla[9].sum().round()
total_oct = df_tabla[11].sum().round()
total_nov = df_tabla[11].sum().round()
total_dic = df_tabla[12].sum().round()  
total_ene = df_tabla[1].sum().round()  
total_feb = df_tabla[2].sum().round()  
total_mar = df_tabla[3].sum().round()  

# Crear una fila adicional para cada columna con los totales
total_row = ['Total', total_ago, total_sep, total_oct, total_nov, total_dic, total_ene, total_feb, total_mar]

# Agregar la fila de totales debajo de cada columna
procesos = df_tabla.index.tolist()
datos_tabla = [procesos] + [df_tabla[col].tolist() + [total_row[i+1]] for i, col in enumerate([8, 9, 11, 11, 12, 1, 1, 3])]

# Crear la tabla
fig = go.Figure(data=[go.Table(
    header=dict(values=['Proceso','Ago', 'Sep','Oct', 'Nov', 'Dic', 'Ene', 'Feb', 'Mar']),
    cells=dict(values=datos_tabla,
               align=['left','center', 'center', 'center', 'center', 'center']))])

# Ajustar el diseño y formato
fig.update_layout(
    title='Datos de Producción por Máquina y Mes (tn)',
    font=dict(size=11),  # Ajustar el tamaño del texto
    template='plotly_white'  # Cambiar el tema a plotly_white
)

# Mostrar la tabla
fig.show()


In [26]:
# Calcular los totales por mes
totals = {
    'Ago': df_tabla[8].sum().round(2),
    'Sep': df_tabla[9].sum().round(2),
    'Oct': df_tabla[10].sum().round(2),
    'Nov': df_tabla[11].sum().round(2),
    'Dic': df_tabla[12].sum().round(2),
    'Ene': df_tabla[1].sum().round(2),
    'Feb': df_tabla[2].sum().round(2),
    'Mar': df_tabla[3].sum().round(2),
}

# Calcular la mediana de los totales por mes
median_total = np.median(list(totals.values())).round(2)

# Crear el gráfico de barras con los totales por mes y mostrar los valores dentro de las barras
fig = go.Figure(data=[
    go.Bar(
        name='Total Producción', 
        x=list(totals.keys()), 
        y=list(totals.values()), 
        marker=dict(color='#FF7705'),
        text=list(totals.values()),
        textposition='inside',
        textfont=dict(color='white')  # Color del texto dentro de las barras
    )
])

# Añadir una línea que represente la mediana de la producción
fig.add_trace(go.Scatter(
    x=list(totals.keys()), 
    y=[median_total] * len(totals), 
    mode='lines',
    name='Mediana de Producción: 593 tn',
    line=dict(color='gray', dash='dash')
))

# Ajustar el diseño y formato del gráfico
fig.update_layout(
    title='Producción Total por Mes',
    xaxis_title='Mes',
    yaxis_title='Producción Total (tn)',
    font=dict(size=11),
    template='plotly_white'
)

# Mostrar el gráfico
fig.show()


In [27]:
# Agrupar por 'Proceso' y 'dow' y calcular el promedio de 'Produccion'
df_dia = df_producion.groupby(['Proceso', 'dow'])['Producion'].mean().reset_index()

In [28]:
"""
import plotly.graph_objects as go

# Supongamos que df_ea es tu DataFrame original

# Lista de días de la semana en el orden deseado
dias_ordenados = ['lunes', 'martes', 'miércoles', 'jueves', 'viernes', 'sábado', 'domingo', 'festivo']
colores_Celsia = ['#f37620', '#585a5b', '#fec431', '#1fa1db', '#00be91', '#ca1e48', '#19459a', '#ef966e', '#949495', '#fae364']
# Crear una lista de Box para cada día de la semana
traces = []
for dia in dias_ordenados:
    trace = go.Box(
        y=df_dia[df_dia['dow'] == dia]['Producion'],
        boxpoints='all',
        name=dia.capitalize(),
        marker_color=colores_Celsia[dias_ordenados.index(dia)]  # Asignar un color correspondiente
    )
    traces.append(trace)

# Crear el layout
layout = go.Layout(
    title='Producción por día [tn]',
    xaxis=dict(title='Día de la Semana'),
    yaxis=dict(title='Producción [tn]'),
)

# Crear la figura
fig = go.Figure(data=traces, layout=layout)

# Mostrar la figura
fig.show()
"""

"\nimport plotly.graph_objects as go\n\n# Supongamos que df_ea es tu DataFrame original\n\n# Lista de días de la semana en el orden deseado\ndias_ordenados = ['lunes', 'martes', 'miércoles', 'jueves', 'viernes', 'sábado', 'domingo', 'festivo']\ncolores_Celsia = ['#f37620', '#585a5b', '#fec431', '#1fa1db', '#00be91', '#ca1e48', '#19459a', '#ef966e', '#949495', '#fae364']\n# Crear una lista de Box para cada día de la semana\ntraces = []\nfor dia in dias_ordenados:\n    trace = go.Box(\n        y=df_dia[df_dia['dow'] == dia]['Producion'],\n        boxpoints='all',\n        name=dia.capitalize(),\n        marker_color=colores_Celsia[dias_ordenados.index(dia)]  # Asignar un color correspondiente\n    )\n    traces.append(trace)\n\n# Crear el layout\nlayout = go.Layout(\n    title='Producción por día [tn]',\n    xaxis=dict(title='Día de la Semana'),\n    yaxis=dict(title='Producción [tn]'),\n)\n\n# Crear la figura\nfig = go.Figure(data=traces, layout=layout)\n\n# Mostrar la figura\nfig.show(

In [36]:
fig = px.bar(df_dia_2, x='dow', y='Producion', barmode='stack',
             labels={'dow': 'Día de la Semana', 'Producion': 'Producción (Ton)'},
             title='Producción promedio por Día de la Semana - Planta General')

# Mostrar el gráfico
fig.show()

El análisis de la producción diaria revela que los viernes tienen el mayor promedio de producción en comparación con otros días de la semana. La producción disminuye significativamente durante los fines de semana y días festivos. Tras el viernes, el martes registra la segunda mayor producción, mientras que lunes, miércoles y jueves muestran niveles de producción similares.

In [30]:
# Convertir la columna 'dow' a tipo categórico y ordenarla
df_dia['dow'] = pd.Categorical(df_dia['dow'], categories=dias_ordenados, ordered=True)


# Crear el gráfico de barras apiladas
fig = px.bar(df_dia, x='dow', y='Producion', color='Proceso', barmode='stack',
             labels={'dow': 'Día de la Semana', 'Producion': 'Producción (toneladas)'},
             title='Producción por Día de la Semana y Proceso')

# Mostrar el gráfico
fig.show()

Para la mayor producción los viernes es la contribución del proceso de Espuma, que representa el 36% de la producción total. Este proceso aporta significativamente más los viernes en comparación con el resto de los días.